In [3]:
from drain.model import Drain,Node,LogCluster
from drain.optimizer import Optimizer
from drain.plotter import createPlot
from drain.partition import Partition
from utils.data_utils import *
from unsupervised.model import Model,train
from unsupervised.predictor import count_metries
from torch import nn,optim
from torch.utils.data import  DataLoader
import pandas as pd
from unsupervised.wf_constructor import workflow_constructor
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 32
batch_size = 1024
input_size = 1
window_size = 10
num_layers = 2
hidden_size = 64
'''
parse
logPath: HDFS.log
parsed_result:logClusters;item_to_event_id.csv

partition
label_file:anoramly_label.csv
parsed_result:item_to_event_id.csv
instance_file: hdfs_instance
output:normaly.csv/abnormal.csv

split_data:
normaly:train,test_normal,test_abnormal

unsupervised:
input:train,test_(ab)normal
output:model
'''
# parse
logPath = 'D:\\毕业设计\\loghub\\HDFS_1\\HDFS.log'
parsed_result = '.\\data\\parser\\log_item_to_label.csv'
cluster_result = '.\\data\\parser\\'
# partition
partition_output = '.\\data\\unsupervised\\dataset\\'
instance_file = 'instance.csv'
label_file = '.\\data\\partition\\anormaly_label.csv'
normal_output = 'normal.csv'
abnormal_output = 'abnormal.csv'
lstm_dataset = '.\\data\\unsupervised\\dataset\\'
# instance_file: hdfs_instance
# output:normaly.csv/abnormal.csv
#
# split_data:
# normaly:train,test_normal,test_abnormal
#
# unsupervised:
# input:train,test_(ab)normal
# output:model

In [10]:
'''
parsing phase
'''
def parse_log(logPath,parsed_result):

    rex = ['blk_(|-)[0-9]+', '(/|)([0-9]+\.){3}[0-9]+(:[0-9]+|)(:|)']
    removeCol = [0,1,2]
    myParser = Drain(rex=rex,removeCol=removeCol,st=0.5)
    myParser.fit(isReconstruct=True,inputFile=logPath,outputFile=parsed_result)

#     myParser.save(savePath=cluster_result)


In [11]:
parse_log('data/logs/HDFS/HDFS_2k.log','data/logs/result/hdfs.csv')

load data: 2000it [00:00, 6644.67it/s]


build the prefix tree process takes 0.3579902648925781
the number of log used to build the parser is  2000
*********************************************


In [12]:
from drain.evaluator import evaluate

In [13]:
evaluate('data/logs/result/hdfs.csv','data/logs/HDFS/HDFS_2k.log_structured.csv')

Precision: 1.0000, Recall: 1.0000, F1_measure: 1.0000, Parsing_Accuracy: 0.9970


(0.9999901893648792, 0.997)

In [14]:
parse_log('data/logs/Apache/Apache_2k.log','data/logs/result/apache.csv')

load data: 2000it [00:00, 6501.99it/s]


build the prefix tree process takes 0.32255101203918457
the number of log used to build the parser is  2000
*********************************************


In [15]:
evaluate('data/logs/result/apache.csv','data/logs/Apache/Apache_2k.log_structured.csv')

Precision: 1.0000, Recall: 1.0000, F1_measure: 1.0000, Parsing_Accuracy: 1.0000


(1.0, 1.0)